In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fuzzywuzzy

In [ ]:
import pandas as pd

briefs_df = pd.read_csv("/content/drive/MyDrive/LLM/extracted_briefs+toc_3_with_first_1000_extractions.csv")

In [ ]:
print(briefs_df.head)

<bound method NDFrame.head of        Unnamed: 0.1  Unnamed: 0  \
0               0.0         1.0   
1               1.0         2.0   
2               2.0         3.0   
3               3.0         4.0   
4               4.0         5.0   
...             ...         ...   
14576       14576.0     14577.0   
14577       14577.0     14578.0   
14578       14578.0     14579.0   
14579       14579.0     14580.0   
14580           NaN         NaN   

                                                    text created_timestamp  \
0      b'No. 21-2\nIN THE\n\nSupreme Court of the Uni...        08-02-2021   
1      b'No. 21-2\nIN THE\n\nSupreme Court of the Uni...        08-16-2021   
2      b"No. _____\n\nIn the\nSupreme Court of the Un...        06-30-2021   
3      b"No. _______\nIn The\n\nSupreme Court of the ...        07-01-2021   
4      b'No. ___-______\n\nIn The\n\nSupreme Court of...        07-01-2021   
...                                                  ...               ...   
145

In [ ]:
test_text = briefs_df.iloc[31]["One Shot Prompt"]
print(test_text)

I. The Decision Below Deepens a Three-Way Circuit Split on Assessing the Adequacy of an Alternative Forum     A. The Petition Presents A Frequently Recurring And Important Issue     B. The Circuit Split Is Intolerable And Requires Action By The Court II. This Court Should Grant Certiorari To Resolve The Circuit Conflicts     A. The Third Circuit's Decision Is Wrong And Would Deny Petitioners Their Day in Court  III. This Court Should Grant Certiorari Because The Third Circuit's Decision Is Wrong And Would Deny Petitioners Their Day in Court     A. The Third Circuit Did Not, and Its Rule Does Not, Ensure Defendant's Chosen Forum is Fair     B. The Third Circuit's Rule Conflicts With The Doctrine's History And This Court's General Personal Jurisdiction Caselaw, Which Guarantees Parties At Least One Fair Forum: Where The Defendant Is At Home     C. Dismissals Threaten To Disrupt Comity Between The United States And Other Nations     D. Dismissal From a Defendant's Home Forum To A Forum Ri

In [ ]:
print(briefs_df.iloc[31]["url"])

http://www.supremecourt.gov/DocketPDF/21/21-33/183422/20210708154130377_newmont%20Cert%20Petition%20final.pdf


In [ ]:
'''
# This should no longer be necessary since preprocessing with regex now splits the headers into a list
import re

formatted_test = re.sub(r'(?<!^)(?=\b(?:[IVXLCDM]+|\b[A-J]|\d)\.)', '\n', test_text) # Search only A-J to limit false positives for things like U.S.
print(formatted_test)

I. The Decision Below Deepens a Three-Way Circuit Split on Assessing the Adequacy of an Alternative Forum     
A. The Petition Presents A Frequently Recurring And Important Issue     
B. The Circuit Split Is Intolerable And Requires Action By The Court 
II. This Court Should Grant Certiorari To Resolve The Circuit Conflicts     
A. The Third Circuit's Decision Is Wrong And Would Deny Petitioners Their Day in Court  
III. This Court Should Grant Certiorari Because The Third Circuit's Decision Is Wrong And Would Deny Petitioners Their Day in Court     
A. The Third Circuit Did Not, and Its Rule Does Not, Ensure Defendant's Chosen Forum is Fair     
B. The Third Circuit's Rule Conflicts With The Doctrine's History And This Court's General Personal Jurisdiction Caselaw, Which Guarantees Parties At Least One Fair Forum: Where The Defendant Is At Home     
C. Dismissals Threaten To Disrupt Comity Between The United States And Other Nations     
D. Dismissal From a Defendant's Home Forum To A

In [ ]:
test_content = briefs_df.iloc[31]["content"]
print(test_content)

TABLE OF AUTHORITIES
Adelson v. Hananel,
510 F.3d 43 (1st Cir. 2007) .................................... 18
Aguinda v. Texaco, Inc.,
303 F. 3d 470 (2d Cir. 2002) .................................. 36
Atlantic Marine Construction Co. v. United
States District Court,
571 U.S. 49 (2013) ............................................ 16, 18
Bank of Credit & Commerce International
(Overseas) Ltd. v. State Bank of Pakistan
(âBCCIâ),
273 F.3d 241 (2d Cir. 2001) ......................... 3, 19, 20
Baris v. Sulpicio,
932 F.2d 1540 (5th Cir. 1991) ................................ 19
Bhatnagar by Bhatnagar v. Surrendra Overseas,
52 F.3d 1220 (3d Cir. 1995) ................................... 34
Canadian Overseas Ores, Ltd. v. Compania de
Acero del Pacifico S.A.,
528 F. Supp. 1337 (S.D.N.Y. 1982) ................. 17 n. 2
Carijano v. Occidental Petroleum Corp.,
643 F.3d 1216 (9th Cir. 2011) .......................... 31, 35
Chavez v. Dole Food Co.,
836 F.3d 205 (3d Cir. 2016) ..................

In [ ]:
cleaned_test_content = re.sub(r'(?<![\.\?!])\n(?!\n)', ' ', test_content)
print(cleaned_test_content)

TABLE OF AUTHORITIES Adelson v. Hananel, 510 F.3d 43 (1st Cir. 2007) .................................... 18 Aguinda v. Texaco, Inc., 303 F. 3d 470 (2d Cir. 2002) .................................. 36 Atlantic Marine Construction Co. v. United States District Court, 571 U.S. 49 (2013) ............................................ 16, 18 Bank of Credit & Commerce International (Overseas) Ltd. v. State Bank of Pakistan (âBCCIâ), 273 F.3d 241 (2d Cir. 2001) ......................... 3, 19, 20 Baris v. Sulpicio, 932 F.2d 1540 (5th Cir. 1991) ................................ 19 Bhatnagar by Bhatnagar v. Surrendra Overseas, 52 F.3d 1220 (3d Cir. 1995) ................................... 34 Canadian Overseas Ores, Ltd. v. Compania de Acero del Pacifico S.A., 528 F. Supp. 1337 (S.D.N.Y. 1982) ................. 17 n. 2 Carijano v. Occidental Petroleum Corp., 643 F.3d 1216 (9th Cir. 2011) .......................... 31, 35 Chavez v. Dole Food Co., 836 F.3d 205 (3d Cir. 2016) ..................

In [ ]:
import re
from fuzzywuzzy import process, fuzz

def preprocess_text(text):
    # Normalize single newlines following sentence-ending punctuation, ignore double newlines
    processed_text = re.sub(r'(?<=[.!?])\s*\n(?!\n)', '\n', text)
    processed_text = re.sub(r'(?<![\.\?!])\n(?!\n)', ' ', processed_text)
    processed_text = re.sub(r'(?<=[.!?])\s*\n\n', '\n\n', processed_text)
    return processed_text.strip()

def match_headers_to_sections(headers, content, threshold=70):
    lines = content.split('\n')
    matched_sections = {}
    section_starts = []

    # Find match points for headers
    for header in headers:
        high_score = 0
        best_match = None
        for line in lines:
            score = fuzz.token_sort_ratio(header, line)
            if score > high_score:
                high_score = score
                best_match = line
            if score > threshold:
                break  # Assuming line order in content follows headers logically
        print(f"The header is: {header}")
        print(f"Best match is: {best_match}")
        print(f"The score is: {high_score}")
        if best_match and high_score > threshold:
            match_index = lines.index(best_match)
            section_starts.append(match_index)
            matched_sections[header] = best_match

    # Create sections from matches
    sections = []
    for i in range(len(section_starts)):
        start_idx = section_starts[i]
        end_idx = section_starts[i + 1] if i + 1 < len(section_starts) else len(lines)
        section_content = "\n".join(lines[start_idx:end_idx])
        sections.append((headers[i], section_content))

    return sections

# Example headers and content
headers_text = formatted_test
content_text = cleaned_test_content

# Preprocess the headers and content
headers = headers_text.split('\n')
content = preprocess_text(content_text)

# Match headers to content and split into sections
sections = match_headers_to_sections(headers, content_text, threshold=90)
'''
for header, section in sections:
    print(f"Header: {header}\nSection: {section}\n")
'''

The header is: I. The Decision Below Deepens a Three-Way Circuit Split on Assessing the Adequacy of an Alternative Forum     
Best match is: I. The Decision Below Deepens a Three-Way Circuit Split on Assessing the Adequacy of an Alternative Forum.
The score is: 100
The header is: A. The Petition Presents A Frequently Recurring And Important Issue     
Best match is:  23 A. The Petition Presents A Frequently Recurring And Important Issue.
The score is: 98
The header is: B. The Circuit Split Is Intolerable And Requires Action By The Court 
Best match is: B. The Circuit Split Is Intolerable And Requires Action By The Court.
The score is: 100
The header is: II. This Court Should Grant Certiorari To Resolve The Circuit Conflicts     
Best match is: II. This Court Should Grant Certiorari To Resolve The Circuit Conflicts.
The score is: 100
The header is: A. The Third Circuit's Decision Is Wrong And Would Deny Petitioners Their Day in Court  
Best match is: III. This Court Should Grant Certio

'\nfor header, section in sections:\n    print(f"Header: {header}\nSection: {section}\n")\n'

In [ ]:
test_idx = 25

In [ ]:
test_text = briefs_df.iloc[test_idx]["One Shot Prompt"]

formatted_test = re.sub(r'(?<=\S) {2,}', '\n', test_text)
print(formatted_test)

A Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
1. Petitioners forfeited any argument that preemption cannot support Rule 12(b)(6) dismissal of a manufacturing defect claim, and there is no circuit split on this issue.
2. Petitioners do not identify a controlling split in authority regarding the dismissal of their failure to warn claims under Twombly and Iqbal.
B The decisions below are correct.
1. The Ninth Circuit properly affirmed dismissal of Petitioners' failure to warn claims.
2. The Ninth Circuit properly dismissed Petitioners' manufacturing defect claims, and Petitioners forfeited any claim of error on this point.
C This is a poor vehicle for addressing the question presented.


In [ ]:
print(briefs_df.iloc[test_idx]["url"])

http://www.supremecourt.gov/DocketPDF/21/21-26/195996/20211008201923739_41395%20pdf%20Rawlin.pdf


In [ ]:
test_content = briefs_df.iloc[test_idx]["content"]
print(test_content)

TABLE OF AUTHORITIESâContinued
Page
21 C.F.R. Â§ 803.9 .........................................................23
21 C.F.R. Â§ 803.9(a) .....................................................22
21 C.F.R. Â§ 814.82 .......................................................10
21 C.F.R. Â§ 814.82(a)(2).................................................6
21 C.F.R. Â§ 814.84(b)(2).................................................7
21 C.F.R. Â§ 878.3530 .....................................................6
OTHER AUTHORITIES
5B Charles Alan Wright & Arthur R. Miller, FEDERAL PRACTICE AND PROCEDURE Â§ 1357 (3d ed.
2004 & Supp. 2014) .................................................27
72 Fed. Reg. 15,855 (Apr. 9, 2007) Notices .................10
FDA, Medical Device Reports for Systemic
Symptoms in Women with Breast Implants
(Aug. 20, 2020) .........................................................29
Jack B. Weinstein, Preliminary Reflections on
Administration of Complex Litigations, 2009
CARDOZO L.

In [ ]:
# Example headers and content
headers_text = formatted_test
content_text = test_content

# Preprocess the headers and content
headers = headers_text.split('\n')
content = preprocess_text(content_text)

# Match headers to content and split into sections
sections = match_headers_to_sections(headers, content, threshold=90)

The header is: A Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
Best match is:  15 ARGUMENT A. Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
The score is: 96
The header is: 1. Petitioners forfeited any argument that preemption cannot support Rule 12(b)(6) dismissal of a manufacturing defect claim, and there is no circuit split on this issue.
Best match is: 1. Petitioners forfeited any argument that preemption cannot support Rule 12(b)(6) dismissal of a manufacturing defect claim, and there is no circuit split on this issue.
The score is: 100
The header is: 2. Petitioners do not identify a controlling split in authority regarding the dismissal of their failure to warn claims under Twombly and Iqbal.
Best match is: 2. Petitioners do not identify a controlling 

In [ ]:
for index, section in enumerate(sections):
  print(f"Section number {index+1}:")
  print(section[1])

Section number 1:
 15 ARGUMENT A. Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
Petitioners concede the narrow gap between express and implied preemption, which has been recognized âby the federal judiciary nearly 200 timesâ since 2009, is by Congressional design. Pet. 20â21 (noting the Tenth Circuit's observation âthat the concept of federal preemption of state law claims in relation to medical devices first appeared when Congress enacted the MDA in 1976â). While Petitioners may find it â âdifficult to believe that Congress would' â preempt state law tort claims by consumers alleging injury from âFDA-approved devices . . . this is exactly what a pre-emption clause for medical devices does by its terms.â Riegel, 552 U.S. at 326 (majority opinion, quoting Ginsburg, J., dissenting). As the Riegel majority explained, the preemptive effect of the MDA was 

In [ ]:
# This does essentially the same thing as match_headers_to_sections but it also deletes the header from the section to avoid data leaks between train and reference
def new_match_headers_to_sections(headers, content, threshold=70):
    lines = content.split('\n')
    matched_sections = {}
    section_starts = []

    # Find match points for headers
    for header in headers:
        high_score = 0
        best_match = None
        for line in lines:
            score = fuzz.token_sort_ratio(header, line)
            if score > high_score:
                high_score = score
                best_match = line
            if score > threshold:
                break  # Assuming line order in content follows headers logically
        print(f"The header is: {header}")
        print(f"Best match is: {best_match}")
        print(f"The score is: {high_score}")
        if best_match and high_score > threshold:
            match_index = lines.index(best_match)
            section_starts.append(match_index)
            matched_sections[header] = best_match

    # Create sections from matches
    sections = []
    for i in range(len(section_starts)):
        start_idx = section_starts[i]
        end_idx = section_starts[i + 1] if i + 1 < len(section_starts) else len(lines)
        section_content = "\n".join(lines[start_idx:end_idx])
        # Match the last 4 words in the header to the section to find its end, and slice off the header.
        # Use regex to find the position of the last four words of the header in the section_content
        last_four_words = ' '.join(headers[i].split()[-4:])  # Get last four words of the header
        # print(f"The last 4 words are: {last_four_words}")
        regex_pattern = re.escape(last_four_words) + r'.*?(?=\n|$)'  # Regex to find these words followed by anything until a newline or end of string
        match = re.search(regex_pattern, section_content, re.DOTALL)  # DOTALL to make '.' match newlines as well
        # print(f"The match is {match}")
        if match:
            # Update section_content to start after the matched header
            section_content = section_content[match.end():].strip()  # Start after the end of the match
        # Replace empty or whitespace-only strings with None
        if not section_content.strip():
            section_content = None
        sections.append({headers[i]: section_content})

    # sections_json = json.dumps(sections, indent=4) # Possibly return a JSON instead
    return sections

In [ ]:
# Example headers and content
headers_text = formatted_test
content_text = test_content

# Preprocess the headers and content
headers = headers_text.split('\n')
content = preprocess_text(content_text)

# Match headers to content and split into sections
new_sections = new_match_headers_to_sections(headers, content, threshold=90)

The header is: A Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
Best match is:  15 ARGUMENT A. Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
The score is: 96
The header is: 1. Petitioners forfeited any argument that preemption cannot support Rule 12(b)(6) dismissal of a manufacturing defect claim, and there is no circuit split on this issue.
Best match is: 1. Petitioners forfeited any argument that preemption cannot support Rule 12(b)(6) dismissal of a manufacturing defect claim, and there is no circuit split on this issue.
The score is: 100
The header is: 2. Petitioners do not identify a controlling split in authority regarding the dismissal of their failure to warn claims under Twombly and Iqbal.
Best match is: 2. Petitioners do not identify a controlling 

In [ ]:
for index, section in enumerate(new_sections):
  print(f"Section number {index+1}:")
  for header, content in section.items():
    print(f"Header: {header}\nContent: {content}\n")

Section number 1:
Header: A Petitioners do not identify a circuit split regarding the application of Twombly and Iqbal to express and implied preemption of claims regarding medical devices.
Content: Petitioners concede the narrow gap between express and implied preemption, which has been recognized âby the federal judiciary nearly 200 timesâ since 2009, is by Congressional design. Pet. 20â21 (noting the Tenth Circuit's observation âthat the concept of federal preemption of state law claims in relation to medical devices first appeared when Congress enacted the MDA in 1976â). While Petitioners may find it â âdifficult to believe that Congress would' â preempt state law tort claims by consumers alleging injury from âFDA-approved devices . . . this is exactly what a pre-emption clause for medical devices does by its terms.â Riegel, 552 U.S. at 326 (majority opinion, quoting Ginsburg, J., dissenting). As the Riegel majority explained, the preemptive effect of the MDA wa